<a href="https://colab.research.google.com/github/JeanRintoul/Covid-19/blob/master/Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import pandas as pd
confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
data = pd.read_csv(confirmed) 
# print(data.head())
print (data)



            Province/State  Country/Region      Lat  ...  3/5/20  3/6/20  3/7/20
0                    Anhui  Mainland China  31.8257  ...     990     990     990
1                  Beijing  Mainland China  40.1824  ...     418     422     426
2                Chongqing  Mainland China  30.0572  ...     576     576     576
3                   Fujian  Mainland China  26.0789  ...     296     296     296
4                    Gansu  Mainland China  36.0611  ...     102     119     120
..                     ...             ...      ...  ...     ...     ...     ...
220      Pierce County, WA              US  47.0676  ...       0       0       1
221    Plymouth County, MA              US  42.1615  ...       0       1       1
222  Santa Cruz County, CA              US  36.9741  ...       0       0       1
223       Tulsa County, OK              US  36.1593  ...       0       0       1
224  Montgomery County, TX              US  30.3213  ...       0       0       0

[225 rows x 50 columns]
